In [ ]:
import pandas as pd
from IPython.display import display

meeting_note_csv_path = './audio/싼기타_비싼기타_final.csv'
df_rttm = pd.read_csv(meeting_note_csv_path, sep='|')
display(df_rttm)

In [ ]:
name_dict = {
    "SPEAKER_00" : "AI" ,
    "SPEAKER_01" : "김상원"
}

df_rttm["name"] = df_rttm["speaker_id"].apply(lambda x: name_dict[x])

display(df_rttm)

In [ ]:
meeting_note_txt = df_rttm[['start','end','name','text']].to_json(orient='records', force_ascii=False)
print(meeting_note_txt)

In [ ]:
system_prompt = f'''
    너는 회의 내용을 요약하는 봇이다. 아래 회의록을 읽고 , 주요 내용을 요약하라
    결과는 마크다운 형식으로 작성한다.
    
    아래 형식에 맞추어 작성하라
    
    # 회의 제목
    
    ## 주요 내용
    
    ## 참석자별
    
    ## 결정사항
    
    ========= 이하 회의록 ======= 
    {meeting_note_txt}
'''

from openai import OpenAI, api_key
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')
client = OpenAI(api_key=api_key)

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": system_prompt}
    ],
    temperature=0.1
)

summary = response.choices[0].message.content
summary = summary.strip() #좌우 공백 제거

print(summary)

with open('audio/guitar_sumaary.md',"w", encoding="utf-8") as f:
    f.write(summary)